In [3]:
import numpy as np
import torch
from torch import nn
from torchvision import datasets
from torchvision import transforms
from torch.utils.data import DataLoader

In [4]:
# 固定亂數種子
torch.cuda.manual_seed_all(1)

In [5]:
# 數據預處理
transform = transforms.Compose([
#     transforms.RandomResizedCrop(224),# 對圖像進行隨機裁減
#     transforms.RandomRotation(20),# 對圖像進行隨機旋轉
#     transforms.RandomHorizontalFlip(p=0.5),# 對圖像進行隨機水平翻轉
    transforms.ToTensor()# 變成tensor格式
]) # 數據增強

# 讀取數據
root = "./data/MNIST"
train_data = datasets.ImageFolder(root + "/train",transform)
test_data = datasets.ImageFolder(root + "/test",transform)

In [6]:
batch_size = 256
train_dataloader = DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(dataset=test_data, batch_size=batch_size, shuffle=True)

In [7]:
classes = train_data.classes
classes_index = train_data.class_to_idx
print(classes)
print(classes_index)

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
{'0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9}


In [8]:
# 測試CUDA能否使用
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [9]:
# 轉換成CUDA能讀的格式
# generate data in parallel with PyTorch
for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([256, 3, 28, 28])
Shape of y: torch.Size([256]) torch.int64


In [10]:
# 定義網路結構
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()# 初始化
        # nn.Conv2d(1,32,5,1,2): 通道數，輸出，捲積，步長，在圖片旁邊補0(幾圈)，激勵函數，最大池化2*2
        self.conv1 = nn.Sequential(nn.Conv2d(3,32,5,1,2),nn.ReLU(),nn.MaxPool2d(2,2))
        self.conv2 = nn.Sequential(nn.Conv2d(32,64,5,1,2),nn.ReLU(),nn.MaxPool2d(2,2))
        self.fc1 = nn.Sequential(nn.Linear(64*7*7,500),nn.Dropout(p=0.5),nn.Sigmoid()) #64個特徵圖 大小7*7  輸出500個特徵圖
        self.fc2 = nn.Sequential(nn.Linear(500,10),nn.Softmax(dim=1)) # 輸出10類 [0~9}
        
    def forward(self,x):
         # torch.Size([64, 1, 28, 28]) # 捲積中需要傳入4維 批次大小 圖像通道數 圖片大小
        x = self.conv1(x)
        x = self.conv2(x)
        # torch.Size([64, 1, 28, 28]) -> (64,784)
        x = x.view(x.size()[0],-1) # 4維變2維（在全連接層做計算只能2維）
        x = self.fc1(x)
        x = self.fc2(x)
        return x


In [11]:
model = Net().to(device)
model

Net(
  (conv1): Sequential(
    (0): Conv2d(3, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Sequential(
    (0): Linear(in_features=3136, out_features=500, bias=True)
    (1): Dropout(p=0.5, inplace=False)
    (2): Sigmoid()
  )
  (fc2): Sequential(
    (0): Linear(in_features=500, out_features=10, bias=True)
    (1): Softmax(dim=1)
  )
)

In [12]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())
 
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
 
        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)
 
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
 
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
 
 
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [13]:
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")
torch.save(model.state_dict(),"MNIST_C3_S.pth") # 保存模型

Epoch 1
-------------------------------
loss: 2.302595  [    0/60000]
loss: 1.745272  [25600/60000]
loss: 1.654800  [51200/60000]
Test Error: 
 Accuracy: 97.0%, Avg loss: 1.492439 

Epoch 2
-------------------------------
loss: 1.502183  [    0/60000]
loss: 1.485266  [25600/60000]
loss: 1.475173  [51200/60000]
Test Error: 
 Accuracy: 98.7%, Avg loss: 1.475368 

Epoch 3
-------------------------------
loss: 1.492471  [    0/60000]
loss: 1.492556  [25600/60000]
loss: 1.471551  [51200/60000]
Test Error: 
 Accuracy: 99.1%, Avg loss: 1.470919 

Epoch 4
-------------------------------
loss: 1.469464  [    0/60000]
loss: 1.492714  [25600/60000]
loss: 1.472115  [51200/60000]
Test Error: 
 Accuracy: 99.2%, Avg loss: 1.469687 

Epoch 5
-------------------------------
loss: 1.474232  [    0/60000]
loss: 1.468520  [25600/60000]
loss: 1.469378  [51200/60000]
Test Error: 
 Accuracy: 99.2%, Avg loss: 1.469651 

Epoch 6
-------------------------------
loss: 1.469871  [    0/60000]
loss: 1.464733  [256